In [1]:
#!pip install dateparser
from sqlalchemy import create_engine
import pandas as pd
import re
from dateparser.search import search_dates

In [2]:
df = pd.read_csv('pattern.csv', encoding = 'cp1252')
df.head()

tables = []
for index, row in df.iterrows():
    tables.append(row['tableId'])

print(tables[1])
print(len(tables))

03bfc26a-c6d0-4761-b8f5-47acf2290d02
27


In [3]:
# Importing environmental variables library that reads from the .env file
from dotenv import load_dotenv

# Loading key-value pairs from the .env file into the OS environment
load_dotenv()

# Reading the key-value pairs from the OS environment
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
db_hostname = os.getenv("DB_HOST")
db_name = os.getenv("DB_DATABASE")

# Using those variables in the connection string using "F" strings
conn_string = f"mysql+mysqldb://{user}:{password}@{db_hostname}/{db_name}?charset=utf8mb4"
engine = create_engine(conn_string)

In [4]:
query = "SELECT * FROM issues;"
with engine.connect() as conn:
    df = pd.read_sql(query, conn)
df
df_issue = df[['tableId', 'rowIndex', 'status_txt']]
print(len(df_issue))
df_issue.head()

5190


,tableId,rowIndex,status_txt
0,0114daa6-c048-4081-b4cf-7a9d7461fa44,1,Monitor in 2018.
1,0114daa6-c048-4081-b4cf-7a9d7461fa44,2,Monitor in 2018 and implement weed control as ...
2,0114daa6-c048-4081-b4cf-7a9d7461fa44,3,Remove erosion log.
3,0114daa6-c048-4081-b4cf-7a9d7461fa44,4,Monitor in 2018 and implement weed control as ...
4,0114daa6-c048-4081-b4cf-7a9d7461fa44,5,Remove sediment fence.


In [ ]:
'''#Checking for the rows which might have string length 0
for index, row in df_issue.iterrows():
    if row['tableId'] in tables:
        #print("---------------------", "---------------------", "---------------------", "---------------------", "---------------------")
        #print(row['status_txt'].replace("<s>", ""))
        if len(str(row['status_txt'])) <1:
            print(row['status_txt'])
            print(row['rowIndex'])
            print(row['tableId'])'''

In [5]:
#Counting relevant rows. Rows which belong in the list of tables provided
count = 0
for index, row in df_issue.iterrows():
    try:
        if row['tableId'] in tables:
            count = count +1
    except:
        continue
print(count)

1946


In [14]:
def date_parser_date_count(string):
    ''' Date Finding Function
        Input:  Takes any string as input 
        Output1: Returns the first date identified 
        Output2: Returns the number of dates identified in the string'''
    
    para = search_dates(string, 
                        settings={'STRICT_PARSING': True, 
                                  'DATE_ORDER': 'DMY'})
    date_string = ""
    count = 0
    try:
        for x in para:
            count = count+1
            if date_string == "":
                date_string = x[0]
    except:
        # No date identified or error in finding a date
        "Hello"
        
    return(date_string, count)

line = "July 13, 2016 -- Vegetation is filling in well with desirable species throughout tract."
date_string, date_count  = date_parser_date_count(line)
print(date_string)
print(date_count)

July 13, 2016
1


In [26]:
def split_issues(status_txt):
    parsed_issues = []
    issue_str = ""
    
    date_string = ""
    date_count = 0
    
    status_txt_multiple = status_txt.split(".")
    
    i = 0
    new_issue_flag = 0
    new_issue_with_date_flag = 0 
    
    for line in status_txt_multiple:
        #print("line", line)
        #print("issue_str", issue_str)
        i = i+1
        new_issue_flag = 0
        new_issue_with_date_flag = 0
        if i == 1:
            issue_str = line
            date_string, date_count  = date_parser_date_count(line)
            #dates.append(dates_a[0])
            continue
            
        # usuallu a new issue starts with ". -", 
        # so afer splitting with "." if the first four characters contains - then we can say that it is a new issue
        if "-" in line.strip()[:1]:
            new_issue_flag = 1
            parsed_issues.append(("'" + issue_str + ".").strip())
            issue_str = date_string + " " + line   
            
            
        # a new issue might also start like this "Issue. Sept 17, 2016 - 65-75% vegetation."
        # line 1: Issue.
        # line 2: Sept 17, 2016 - 65-75% vegetation.
        if len(line) > 20:
            date_s, date_c = date_parser_date_count(line)
            #print(date_string)
            if (date_c > 0) and  ("-" in line.strip()[:21]):
                #dates.append(dates_a[0])
                new_issue_with_date_flag = 1
                parsed_issues.append(("'" + issue_str + ".").strip())
                issue_str = line
                date_string = date_s
                date_count = date_c
                
        
        if new_issue_flag == 0 and new_issue_with_date_flag == 0 and len(line) > 0:
            issue_str = issue_str + ". " + line.strip()
                
#         if flag == 1:
#             #print(issue_str)
#             parsed_issues.append(issue_str + ".")
#             issue_str = line
#             flag = 0
        
    if new_issue_with_date_flag ==1:
        parsed_issues.append("'" + issue_str + ".")
        
    elif new_issue_flag ==1:
        parsed_issues.append((date_string + " " + issue_str + ".").strip())
        
    else:
        parsed_issues.append(("'" + issue_str + ".").strip())
    
    return parsed_issues
    

In [28]:
split_issues("July 13, 2016  - No erosion or subsidence issues. Sept 17, 2016 – Removed and disposed of silt fencing. Issue resolved.")

["'July 13, 2016  - No erosion or subsidence issues. Sept 17, 2016 – Removed and disposed of silt fencing. Issue resolved."]

In [27]:
text = "July 19, 2017 - Treed fen on ROW from KP 35+240 to 35+570 appears to be at full functionality. Line of sight barrier existing in wetland at KP 35+500. - Vegetation establishment within wetland is 80-100% compared to off ROW. Dominant vegetation consisting of herbaceous graminoids, buck brush and willows. Hydrophytic species emerging throughout wetland. Higher density of shrubs present on ditchline within wetland due to slightly elevated roach. - Trench crown present in wetland with sufficient amount of breaks for hydrology to be maintained. - Substrate appears stable through high moisture holding capacity and uniform vegetation growth throughout wetland. High organic content present in substrate. - Wetland exhibiting fully saturated conditions. - Wetland exhibiting good potential for wildlife; birds identified using wetland has habitat. - No subsidence or erosion was observed in wetland. - Wetland is stable with appropriate pre-construction contours achieved. Off ROW Black Spruce to emerge onto ROW over time as a tertiary successional species; leave for natural recovery. No net loss of wetland function was observed"
print(text)
print("-----breakdown-----")

for row in split_issues(text):
    print(":   ", row)

July 19, 2017 - Treed fen on ROW from KP 35+240 to 35+570 appears to be at full functionality. Line of sight barrier existing in wetland at KP 35+500. - Vegetation establishment within wetland is 80-100% compared to off ROW. Dominant vegetation consisting of herbaceous graminoids, buck brush and willows. Hydrophytic species emerging throughout wetland. Higher density of shrubs present on ditchline within wetland due to slightly elevated roach. - Trench crown present in wetland with sufficient amount of breaks for hydrology to be maintained. - Substrate appears stable through high moisture holding capacity and uniform vegetation growth throughout wetland. High organic content present in substrate. - Wetland exhibiting fully saturated conditions. - Wetland exhibiting good potential for wildlife; birds identified using wetland has habitat. - No subsidence or erosion was observed in wetland. - Wetland is stable with appropriate pre-construction contours achieved. Off ROW Black Spruce to em

In [21]:
tableId = []
rowIndex = []
status_bin =[]
status_txt = []
issues =[]

none_type = []
for index, row in df_issue.iterrows():
    try:
        if row['tableId'] in tables:
            print("---------------------", "---------------------", "---------------------", "---------------------", "---------------------")
            print(row["status_txt"])
        
            parsed_issues = split_issues(str(row["status_txt"]))
#             print("parsed issues", parsed_issues)
#             print("HELLOOOOO")
            if len(parsed_issues) > 0:
                print("--------------------- breakdown ---------------------")
                for issue in parsed_issues:
                    print("---", issue)
                    
                    tableId.append(row['tableId'])
                    rowIndex.append(row['rowIndex'])
                    status_bin.append(row['status_bin'])
                    status_txt.append(row['status_txt'])
                    issues.append(issue)
                    
            print(row['tableId'])
        else:
            continue
    except:
        none_type.append(index)
        
    

--------------------- --------------------- --------------------- --------------------- ---------------------
June 22, 2016 – Intermittent weeds observed, recommend implementation of control measures. July 13, 2016 - Vegetation continues to establish with desirable species throughout ROW. Diverse mix of vegetation observed. - Cross drains are vegetated and functioning.  - Wetlands stable and consistent with previous assessments. - No erosion or subsidence issues. - KP 0+300- Marker posts and sign down. - Intermittent herbicide application to control scentless chamomile and perennial sow thistle.  - Herbicide application conducted on valve pad and 2m perimeter.  - Was not able to locate or identify any Dalmatian toadflax, 2015 issue resolved. Sept 17, 2016 - 65-75% vegetation established compared to off ROW. - Installed TC sign and marker posts at KP 0+040. - Re-installed marker posts at KP 0+300, issue resolved. - Intermittent herbicide application conducted to control perennial sow th

In [ ]:
df_parsed_issues = pd.DataFrame({'tableId': tableId,
                          'rowIndex' : rowIndex,
                          'status_txt' :  status_txt,
                        'issues': issues})
#len(df_labels)
print(len(df_parsed_issues))

In [ ]:
print(len(none_type))
print(none_type)

In [ ]:
df_parsed_issues.to_csv("parsed_issues.csv", index = False, encoding='utf-8-sig')